## **Install python library auto-sklearn**


In [ ]:
!sudo apt-get install build-essential swig
!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install
!pip install auto-sklearn
!pip install pandas_profiling -U

In [1]:
import json
import requests
import re
import numpy as np
import pandas as pd
import pprint
import datetime
import autosklearn
import pandas_profiling as pp
import pickle
import sklearn.metrics as mtr
from geopy.distance import distance
from sklearn.model_selection import train_test_split
from autosklearn.regression import AutoSklearnRegressor

/usr/local/lib/python3.7/dist-packages/pyparsing.py:3190: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)


## **Collect the Data**

From data.gov.sg api<br>
From January 2017

In [ ]:
ds = None
def addToDataset(data):
  temp = pd.DataFrame(data, columns=data[0].keys())
  global dataset
  if ds is None:
    ds = temp
  else:
    ds = ds.append(temp, ignore_index=True)

url ='https://data.gov.sg'
path = '/api/action/datastore_search?resource_id=42ff9cfe-abe5-4b54-beda-c88f9bb438ee&limit=1000'
while True:
  resp = requests.get(url + path)
  #Convert JSON into Python Object 
  data = json.loads(resp.content)
  if len(data['result']['records']) <= 0:
    break
  
  addToDataset(data['result']['records'])  
  path = data['result']['_links']['next']


ds.to_csv('hdb.csv', index=False)

In [ ]:
def getLatLong(places):
  temp = []
  for place in places:
    query_string = 'https://developers.onemap.sg/commonapi/search?searchVal=' + str(place) + '&returnGeom=Y&getAddrDetails=Y'
    try:
      resp = requests.get(query_string)
      data=json.loads(resp.content)
      if data['found'] > 0:
        temp.append({'name': place, 'latitude':  data["results"][0]["LATITUDE"], 'longitude': data["results"][0]["LONGITUDE"]})
      else:
        print('not found: ' + place)
    except Exception as e: 
      print(str(place))
      print(e)
        
  return pd.DataFrame(temp, columns=temp[0].keys())

## **Get Addresses geo location**

In [ ]:
ds['street_name'] = ds['street_name'].apply(lambda x: x.replace('. ', ' '))
ds['address'] = ds['block'] + ' ' + ds['street_name']
addresses = ds.drop_duplicates(subset='address', keep='first')
addresses = addresses['address'].tolist()
addresses = getLatLong(addresses)
addresses.to_csv('address.csv', index=False)

## **Get MRT geo location**

In [ ]:
mrt = ['Jurong East MRT Station',
'Bukit Batok MRT Station',
'Bukit Gombak MRT Station',
'Choa Chu Kang MRT Station',
'Yew Tee MRT Station',
'Kranji MRT Station',
'Marsiling MRT Station',
'Woodlands MRT Station',
'Admiralty MRT Station',
'Sembawang MRT Station',
'Canberra MRT Station',
'Yishun MRT Station',
'Khatib MRT Station',
'Yio Chu Kang MRT Station',
'Ang Mo Kio MRT Station',
'Bishan MRT Station',
'Braddell MRT Station',
'Toa Payoh MRT Station',
'Novena MRT Station',
'Newton MRT Station',
'Orchard MRT Station',
'Somerset MRT Station',
'Dhoby Ghaut MRT Station',
'City Hall MRT Station',
'Raffles Place MRT Station',
'Marina Bay MRT Station',
'Marina South Pier MRT Station',
'Pasir Ris MRT Station',
'Tampines MRT Station',
'Simei MRT Station',
'Tanah Merah MRT Station',
'Bedok MRT Station',
'Kembangan MRT Station',
'Eunos MRT Station',
'Paya Lebar MRT Station',
'Aljunied MRT Station',
'Kallang MRT Station',
'Lavender MRT Station',
'Bugis MRT Station',
'Tanjong Pagar MRT Station',
'Outram Park MRT Station',
'Tiong Bahru MRT Station',
'Redhill MRT Station',
'Queenstown MRT Station',
'Commonwealth MRT Station',
'Buona Vista MRT Station',
'Dover MRT Station',
'Clementi MRT Station',
'Chinese Garden MRT Station',
'Lakeside MRT Station',
'Boon Lay MRT Station',
'Pioneer MRT Station',
'Joo Koon MRT Station',
'Gul Circle MRT Station',
'Tuas Crescent MRT Station',
'Tuas West Road MRT Station',
'Tuas Link MRT Station',
'Expo MRT Station',
'Changi Airport MRT Station',
'HarbourFront MRT Station',
'Chinatown MRT Station',
'Clarke Quay MRT Station',
'Little India MRT Station',
'Farrer Park MRT Station',
'Boon Keng MRT Station',
'Potong Pasir MRT Station',
'Woodleigh MRT Station',
'Serangoon MRT Station',
'Kovan MRT Station',
'Hougang MRT Station',
'Buangkok MRT Station',
'Sengkang MRT Station',
'Punggol MRT Station',
'Punggol Coast MRT Station',
'Bras Basah MRT Station',
'Esplanade MRT Station',
'Promenade MRT Station',
'Nicoll Highway MRT Station',
'Stadium MRT Station',
'Mountbatten MRT Station',
'Dakota MRT Station',
'MacPherson MRT Station',
'Tai Seng MRT Station',
'Bartley MRT Station',
'Lorong Chuan MRT Station',
'Marymount MRT Station',
'Caldecott MRT Station',
'Bukit Brown MRT Station',
'Botanic Gardens MRT Station',
'Farrer Road MRT Station',
'Holland Village MRT Station',
'one-north MRT Station',
'Kent Ridge MRT Station',
'Haw Par Villa MRT Station',
'Pasir Panjang MRT Station',
'Labrador Park MRT Station',
'Telok Blangah MRT Station',
'Keppel MRT Station',
'Cantonment MRT Station',
'Prince Edward Road MRT Station',
'Bayfront MRT Station',
'Bukit Panjang MRT Station',
'Cashew MRT Station',
'Hillview MRT Station',
'Beauty World MRT Station',
'King Albert Park MRT Station',
'Sixth Avenue MRT Station',
'Tan Kah Kee MRT Station',
'Stevens MRT Station',
'Rochor MRT Station',
'Downtown MRT Station',
'Telok Ayer MRT Station',
'Fort Canning MRT Station',
'Bencoolen MRT Station',
'Jalan Besar MRT Station',
'Bendemeer MRT Station',
'Geylang Bahru MRT Station',
'Mattar MRT Station',
'Ubi MRT Station',
'Kaki Bukit MRT Station',
'Bedok North MRT Station',
'Bedok Reservoir MRT Station',
'Tampines West MRT Station',
'Tampines East MRT Station',
'Upper Changi MRT Station',
'Xilin MRT Station',
'Sungei Bedok MRT Station',
'Woodlands North MRT Station',
'Woodlands South MRT Station',
'Springleaf MRT Station',
'Lentor MRT Station',
'Mayflower MRT Station',
'Bright Hill MRT Station',
'Upper Thomson MRT Station',
'Mount Pleasant MRT Station',
'Napier MRT Station',
'Orchard Boulevard MRT Station',
'Great World MRT Station',
'Havelock MRT Station',
'Maxwell MRT Station',
'Shenton Way MRT Station',
'Marina South MRT Station',
'Gardens by the Bay MRT Station',
'Tanjong Rhu MRT Station',
'Katong Park MRT Station',
'Tanjong Katong MRT Station',
'Marine Parade MRT Station',
'Marine Terrace MRT Station',
'Siglap MRT Station',
'Bayshore MRT Station',
'Bedok South MRT Station'
]

mrt = getLatLong(mrt)
mrt.to_csv('mrt.csv', index=False)

## **Get Shopping Malls geo location**

In [ ]:
malls = [
    '100 AM',
    '313@Somerset',
    'Aperia',
    'Balestier Hill Shopping Centre',
    'Bugis Cube',
    'Bugis Junction',
    'Bugis+',
    'Capitol Piazza',
    'Cathay Cineleisure Orchard',
    'City Gate',
    'City Square Mall',
    'CityLink Mall',
    'Clarke Quay Central',
    'Duo',
    'Far East Plaza',
    'Funan',
    'Great World City',
    'HDB Hub',
    'Holland Village Shopping Mall',
    'ION Orchard',
    'Junction 8',
    'Knightsbridge[1]',
    'Liang Court',
    'Liat Towers',
    'Lucky Plaza',
    'Marina Bay Financial Centre Tower 3',
    'Marina Bay Link Mall',
    'Marina Bay Sands',
    'Marina One',
    'Marina Square',
    'Midpoint Orchard',
    'Millenia Walk',
    'Mustafa Shopping Centre',
    'Ngee Ann City',
    'Orchard Central',
    'Orchard Gateway',
    'Orchard Plaza',
    'Orchard Shopping Centre',
    'Palais Renaissance',
    'Peoples Park Centre',
    'Peoples Park Complex',
    'Plaza Singapura',
    'PoMo',
    'Raffles City',
    'Scotts Square',
    'Serangoon Plaza',
    'Shaw House and Centre',
    'Sim Lim Square',
    'Singapore Shopping Centre',
    'Square 2',
    'Suntec City',
    'Tanglin Mall',
    'Tangs',
    'Tanjong Pagar Centre',
    'Tekka Centre',
    'The Centrepoint',
    'The Paragon',
    'The Poiz [2]',
    'The Shoppes at Marina Bay Sands',
    'The South Beach',
    'Thomson Plaza',
    'United Square, The Kids Learning Mall',
    'Velocity',
    'Wheelock Place',
    'Wisma Atria',
    'Zhongshan Mall',
    '112 Katong',
    'Bedok Mall',
    'Bedok Point',
    'Century Square',
    'Changi Airport',
    'Changi City Point',
    'City Plaza',
    'Djitsun Mall Bedok',
    'Downtown East',
    'East Village',
    'Eastpoint Mall',
    'Elias Mall',
    'Kallang Wave Mall',
    'Katong Square',
    'Katong V',
    'KINEX',
    'Leisure Park Kallang',
    'Loyang Point',
    'Our Tampines Hub',
    'Parkway Parade',
    'Paya Lebar Square',
    'PLQ Mall',
    'Singapore Post Centre',
    'Tampines 1',
    'Tampines Mall',
    'The Flow',
    'White Sands',
    '888 Plaza',
    'Admiralty Place',
    'AMK Hub',
    'Beauty World Centre',
    'Beauty World Plaza',
    'Broadway Plaza',
    'Buangkok Square',
    'Bukit Panjang Plaza',
    'Bukit Timah Plaza',
    'Causeway Point',
    'Compass One',
    'Djitsun Mall',
    'Fajar Shopping Centre',
    'Greenridge Shopping Centre',
    'Greenwich V',
    'Heartland Mall',
    'Hillion Mall',
    'HillV2',
    'Hougang 1',
    'Hougang Green Shopping Mall',
    'Hougang Mall',
    'Jubilee Square',
    'Junction 10',
    'Junction 9',
    'Keat Hong Shopping Centre',
    'KKH The Retail Mall',
    'Limbang Shopping Centre',
    'Lot One',
    'Marsiling Mall',
    'myVillage @ Serangoon',
    'NEX',
    'North East',
    'North West',
    'Northpoint City',
    'Oasis Terraces',
    'Punggol Plaza',
    'Rail Mall',
    'Rivervale Mall',
    'Rivervale Plaza',
    'Sembawang Shopping Centre',
    'Sun Plaza',
    'Sunshine Place',
    'Teck Whye Shopping Centre',
    'The Midtown',
    'The Seletar Mall',
    'Upper Serangoon Shopping Centre',
    'Waterway Point',
    'West Mall',
    'Wisteria Mall',
    'Woodlands Mart',
    'Yew Tee Point',
    'Yew Tee Shopping Centre',
    'Yew Tee Square',
    'Alexandra Retail Centre',
    'HarbourFront Centre',
    'VivoCity',
    '321 Clementi',
    'Alexandra Central',
    'Anchorpoint',
    'Big Box',
    'Boon Lay Shopping Centre',
    'Fairprice Hub',
    'Gek Poh Shopping Centre',
    'Grantral Mall',
    'IMM',
    'JCube',
    'Jem',
    'Jurong Point',
    'OD Mall',
    'Pioneer Mall',
    'Queensway Shopping Centre',
    'Rochester Mall',
    'Taman Jurong Shopping Centre',
    'The Clementi Mall',
    'The Star Vista',
    'Tiong Bahru Plaza',
    'West Coast Plaza',
    'Westgate Mall',
]

malls = getLatLong(malls)
malls.to_csv('malls.csv', index=False)

## **Find shortest distance to mrt and malls**

In [4]:
def findShortestRoute(pos, places):
  dist = []
  for row in zip(places['name'], places['latitude'], places['longitude']):
    dist.append(distance(pos, (row[1], row[2])).km)
  return min(dist)

In [ ]:
sd = {}
for row in zip(addresses['name'], addresses['latitude'], addresses['longitude']):
  sd[row[0]] = {'distMRT': findShortestRoute((row[1], row[2]), mrt), 'distMall': findShortestRoute((row[1], row[2]), malls)}


## **Add the distances to the main dataset**

In [ ]:
def getValue(key, di):
  result = [-1,-1]
  if di.get(key) is not None:
    temp = di[key]
    result = [temp['distMRT'], temp['distMall']]
  return result

In [ ]:
temp = ds.apply(lambda row: pd.Series(getValue(row['block'] + ' ' + row['street_name'], sd), index=['mrt_dist', 'mall_dist']), axis=1)
ds = pd.concat([ds, temp], axis=1)
ds.to_csv('hdb.csv', index=False)

# Regression
## **Load the data**
Upload and read the above data from file

In [3]:
ds = pd.read_csv('hdb.csv', dtype='category')
mrt = pd.read_csv('mrt.csv', dtype=str)
malls = pd.read_csv('malls.csv', dtype=str)
addresses = pd.read_csv('address.csv', dtype=str)
ds

,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,remaining_lease,lease_commence_date,storey_range,_id,block,mrt_dist,mall_dist
0,ANG MO KIO,2 ROOM,Improved,44.0,ANG MO KIO AVE 10,232000.0,2017-01,61 years 04 months,1979,10 TO 12,1,406,0.9814939966679584,1.0030181185550886
1,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 4,250000.0,2017-01,60 years 07 months,1978,01 TO 03,2,108,0.16682829315502534,0.8969421626557303
2,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,262000.0,2017-01,62 years 05 months,1980,01 TO 03,3,602,0.5321547725819701,1.5277222757674624
3,ANG MO KIO,3 ROOM,New Generation,68.0,ANG MO KIO AVE 10,265000.0,2017-01,62 years 01 month,1980,04 TO 06,4,465,0.9164211594265872,1.0292898412156777
4,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,265000.0,2017-01,62 years 05 months,1980,01 TO 03,5,601,0.4984189814658886,1.571443059003091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90118,YISHUN,EXECUTIVE,Maisonette,146.0,YISHUN ST 61,590000.0,2021-01,65 years 08 months,1987,04 TO 06,90119,643,0.6084995010101671,0.5611100219286683
90119,YISHUN,EXECUTIVE,Maisonette,146.0,YISHUN ST 71,588000.0,2021-01,64 years 07 months,1986,10 TO 12,90120,724,0.7285419488763735,0.707678678091571
90120,YISHUN,EXECUTIVE,Apartment,142.0,YISHUN ST 81,670000.0,2021-01,66 years,1988,04 TO 06,90121,834,0.32341712329895045,0.8673191926193174
90121,YISHUN,EXECUTIVE,Apartment,142.0,YISHUN ST 81,690000.0,2021-01,65 years 11 months,1987,07 TO 09,90122,877,0.5090341540527809,0.7869863440341452


## **Preprocessing**

In [ ]:
# convert floor_area_sqm, resale_price, mrt_dist, mall_dist to float
# transform remaining_lease to decimal
# remove columns: street name, month, lease_commence_date, id, block
# delete duplicate records if any
def transformYearStringToNumber(s):
  year = 0
  numbers = re.findall(r"\d+", s)
  if len(numbers) == 1:
    year = float(numbers[0])
  elif len(numbers) >= 2:
    year = float(numbers[0]) + float(numbers[1]) / 12
  
  return year

ds['remaining_lease'] = ds['remaining_lease'].apply(lambda x: transformYearStringToNumber(x))
ds['resale_price_in_million'] = ds['resale_price'].astype('float64') / 1000000  # new value is in million
ds['floor_area_sqm'] = ds['floor_area_sqm'].astype('float64')
ds['mall_dist'] = ds['mall_dist'].astype('float64')
ds['mrt_dist'] = ds['mrt_dist'].astype('float64')

ds = ds.drop(['street_name','month','lease_commence_date','_id','block','resale_price'], axis=1) 
ds = ds.drop_duplicates(ignore_index = True)
ds

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


,town,flat_type,flat_model,floor_area_sqm,remaining_lease,storey_range,mrt_dist,mall_dist,resale_price_in_million
0,ANG MO KIO,2 ROOM,Improved,44.0,61.333333,10 TO 12,0.981494,1.003018,0.232
1,ANG MO KIO,3 ROOM,New Generation,67.0,60.583333,01 TO 03,0.166828,0.896942,0.250
2,ANG MO KIO,3 ROOM,New Generation,67.0,62.416667,01 TO 03,0.532155,1.527722,0.262
3,ANG MO KIO,3 ROOM,New Generation,68.0,62.083333,04 TO 06,0.916421,1.029290,0.265
4,ANG MO KIO,3 ROOM,New Generation,67.0,62.416667,01 TO 03,0.498419,1.571443,0.265
...,...,...,...,...,...,...,...,...,...
89845,YISHUN,EXECUTIVE,Maisonette,146.0,65.666667,04 TO 06,0.608500,0.561110,0.590
89846,YISHUN,EXECUTIVE,Maisonette,146.0,64.583333,10 TO 12,0.728542,0.707679,0.588
89847,YISHUN,EXECUTIVE,Apartment,142.0,66.000000,04 TO 06,0.323417,0.867319,0.670
89848,YISHUN,EXECUTIVE,Apartment,142.0,65.916667,07 TO 09,0.509034,0.786986,0.690


In [ ]:
pp.ProfileReport(ds)

Summarize dataset:   0%|          | 0/22 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

## **Training the model**

In [ ]:
y = ds['resale_price_in_million']
X = ds.drop(['resale_price_in_million'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
# define search
model = AutoSklearnRegressor(time_left_for_this_task=2*60*60, per_run_time_limit=10*60, n_jobs=-1, memory_limit=4096, exclude_estimators=['k_nearest_neighbors', 'mlp', 'decision_tree'], resampling_strategy_arguments= {'train_size': 0.8})
# perform the search
model.fit(X_train, y_train)
# summarize
model.sprint_statistics()


/usr/local/lib/python3.7/dist-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36925 instead
  http_address["port"], self.http_server.port


'auto-sklearn results:\n  Dataset name: ea505f1a-7e5e-11eb-87d1-0242ac1c0002\n  Metric: r2\n  Best validation score: 0.970554\n  Number of target algorithm runs: 106\n  Number of successful target algorithm runs: 78\n  Number of crashed target algorithm runs: 3\n  Number of target algorithms that exceeded the time limit: 13\n  Number of target algorithms that exceeded the memory limit: 12\n'

## **Model Evaluation**

In [ ]:
print(model.show_models())
y_hat = model.predict(X_test)
print("MSE: %.3f" % mtr.mean_squared_error(y_test, y_hat))

[(0.260000, SimpleRegressionPipeline({'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'no_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'no_coalescense', 'data_preprocessing:numerical_transformer:imputation:strategy': 'mean', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'quantile_transformer', 'feature_preprocessor:__choice__': 'no_preprocessing', 'regressor:__choice__': 'gradient_boosting', 'data_preprocessing:numerical_transformer:rescaling:quantile_transformer:n_quantiles': 1494, 'data_preprocessing:numerical_transformer:rescaling:quantile_transformer:output_distribution': 'normal', 'regressor:gradient_boosting:early_stop': 'train', 'regressor:gradient_boosting:l2_regularization': 0.02028139768712157, 'regressor:gradient_boosting:learning_rate': 0.11348206382688412, 'regressor:gradient_boosting:loss': 'least_squares', 'regressor:gradient_boosting:max_bins': 255, 'regressor:gradient_boosting:

## **Save / Load Trained Model**

In [ ]:
#save
with open('autosklearn-reg.pickle', 'wb') as f:
  pickle.dump(model,f)

#load
with open('autosklearn-reg.pickle', 'rb') as f:
  model = pickle.load(f)

## **Make Prediction**

In [ ]:
def makePrediction(req, addresses, mrt, malls, model):
  # calculate the distance
  pos = addresses.loc[addresses['name'] == req['address'], ['latitude','longitude']].values.tolist()[0]
  req['mrt_dist'] = findShortestRoute(pos, mrt)
  req['mall_dist'] = findShortestRoute(pos, malls)
  addr = req.pop('address', None)

  # convert the request to the accepted format
  format_df = pd.DataFrame({'town': pd.Series([], dtype='category'),'flat_type': pd.Series([], dtype='category'), 'flat_model': pd.Series([], dtype='category'), 'floor_area_sqm': pd.Series([], dtype='float64'), 'remaining_lease': pd.Series([], dtype='float64'), 'storey_range': pd.Series([], dtype='category'), 'mrt_dist': pd.Series([], dtype='float64'), 'mall_dist': pd.Series([], dtype='float64')})
  req_df = format_df.append(req, ignore_index=True)
  # somehow after append the datatype is back to object, have to convert again
  req_df['town'] = req_df['town'].astype('category')
  req_df['flat_type'] = req_df['flat_type'].astype('category')
  req_df['flat_model'] = req_df['flat_model'].astype('category')
  req_df['storey_range'] = req_df['storey_range'].astype('category')

  # predict the price
  pred = model.predict(req_df)
  return pred * 1000000


# prepare a data point
req = {'address': '191 BISHAN ST 13', 'town': 'BISHAN', 'flat_type': '5 ROOM', 'flat_model': 'Improved', 'floor_area_sqm': 120.9598, 'remaining_lease': 65.0,	'storey_range': '04 TO 06'}
print("Price Prediction for %s is $%.2f" % (req['address'], makePrediction(req, addresses, mrt, malls, model)))

Price Prediction for 191 BISHAN ST 13 is $754205.84
